**Aim**: Demonstrate the working of the decision tree based ID3 algorithm. Use an appropriate
data set for building the decision tree and apply this knowledge toclassify a new
sample.

---

In [32]:
import numpy as np
from collections import Counter

def entropy(data, target_attribute):
    label_column = [instance[target_attribute] for instance in data]
    _, counts = np.unique(label_column, return_counts=True)
    probabilities = counts / counts.sum()
    entropy_value = -np.sum(probabilities * np.log2(probabilities))
    return entropy_value

In [33]:
def information_gain(data, split_attribute, target_attribute):
    split_attribute_column = [instance[split_attribute] for instance in data]
    _, counts = np.unique(split_attribute_column, return_counts=True)
    probabilities = counts / counts.sum()

    weighted_entropy = 0
    for value, prob in zip(np.unique(split_attribute_column), probabilities):
        subset = [instance for instance in data if instance[split_attribute] == value]
        weighted_entropy += prob * entropy(subset, target_attribute)

    return entropy(data, target_attribute) - weighted_entropy

In [34]:
def find_best_attribute(data, attributes, target_attribute):
    information_gains = [information_gain(data, attribute, target_attribute) for attribute in attributes]
    best_attribute = attributes[np.argmax(information_gains)]
    return best_attribute

In [35]:
def create_decision_tree(data, attributes, target_attribute):
    label_column = [instance[target_attribute] for instance in data]

    # Base cases
    if len(set(label_column)) == 1:
        return label_column[0]
    if len(attributes) == 0:
        return Counter(label_column).most_common(1)[0][0]

    # Recursive case
    best_attribute = find_best_attribute(data, attributes, target_attribute)
    tree = {best_attribute: {}}
    remaining_attributes = [attr for attr in attributes if attr != best_attribute]

    for value in set(instance[best_attribute] for instance in data):
        subset = [instance for instance in data if instance[best_attribute] == value]
        subtree = create_decision_tree(subset, remaining_attributes, target_attribute)
        tree[best_attribute][value] = subtree

    return tree


In [36]:
def classify(instance, tree):
    if isinstance(tree, str):
        return tree
    attribute = list(tree.keys())[0]
    value = instance[attribute]
    subtree = tree[attribute][value]
    return classify(instance, subtree)

In [37]:
import pandas as pd
from pprint import pprint

df = pd.read_csv('playtennis.csv')
data = df.to_dict('records')

target_attribute = 'PlayTennis'
attributes = list(data[0].keys())
attributes.remove(target_attribute)

tree = create_decision_tree(data, attributes, target_attribute)
print("Decision tree:")
pprint(tree)
print()
# # Test instance
df_test = pd.read_csv('playtennis_test.csv')

# Classify test instance
print("Classify test instance:")
df_test['PlayTennis'] = df_test.apply(lambda row: classify(row, tree), axis=1)
df_test

Decision tree:
{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}

Classify test instance:


,PlayTennis,Outlook,Temperature,Humidity,Wind
0,Yes,Rain,Cool,Normal,Weak
1,No,Rain,Mild,High,Strong
